In [12]:
from dataset import FSDDLoader, DataMode
import joblib

In [2]:
loader = FSDDLoader(max_duration=0.9)

C:\Users\adipe\PycharmProjects\Machine-Learning-Final\.venv\Lib\site-packages\librosa\feature\spectral.py:2148: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [13]:
aggregate_data = loader.load(mode=DataMode.AGGREGATE)
flatten_data = loader.load(mode=DataMode.FLATTEN)
sequence_data = loader.load(mode=DataMode.SEQUENCE)

joblib.dump(aggregate_data, "dataset/aggregate_data.pkl")
joblib.dump(flatten_data, "dataset/flatten_data.pkl")
joblib.dump(sequence_data, "dataset/sequence_data.pkl")

['dataset/sequence_data.pkl']